# Zero to thunder

In [5]:
import sys
sys.path.insert(0, '..')

import torch, thunder

from thunder.tests.lit_gpt_model import Config, Block

In [37]:
from lit_gpt.model import Config, LLaMAMLP

In [51]:
cfg = Config.from_name('Llama-2-7b-hf')
with torch.device("cuda"):
    m = LLaMAMLP(cfg)



In [52]:
m

LLaMAMLP(
  (fc_1): Linear(in_features=4096, out_features=11008, bias=False)
  (fc_2): Linear(in_features=4096, out_features=11008, bias=False)
  (proj): Linear(in_features=11008, out_features=4096, bias=False)
)

In [53]:
tm = thunder.jit(m)

In [54]:
tm

ThunderModule(
  (_model): LLaMAMLP(
    (fc_1): Linear(in_features=4096, out_features=11008, bias=False)
    (fc_2): Linear(in_features=4096, out_features=11008, bias=False)
    (proj): Linear(in_features=11008, out_features=4096, bias=False)
  )
)

In [60]:
x = torch.randn(2, 2048, 4096, device="cuda")
(tm(x) - m(x)).abs().max()


tensor(1.4901e-07, device='cuda:0', grad_fn=<MaxBackward1>)

In [61]:
thunder.last_traces(tm)[0]

[# Constructed by Augmented forward pass
 import thunder
 import thunder.core.prims as prims
 import torch
 from thunder.executors.torchex import no_autocast
 
 @torch.no_grad()
 @no_autocast()
 def augmented_forward_fn(input, t_fc_1_weight, t_fc_2_weight, t_proj_weight):
   # input: "cuda:0 f32[2, 2048, 4096]" 
   # t_fc_1_weight: "cuda:0 f32[11008, 4096]" 
   # t_fc_2_weight: "cuda:0 f32[11008, 4096]" 
   # t_proj_weight: "cuda:0 f32[4096, 11008]" 
   t0 = prims.linear(input, t_fc_1_weight, None)  # t0: "cuda:0 f32[2, 2048, 11008]"
   t1 = prims.linear(input, t_fc_2_weight, None)  # t1: "cuda:0 f32[2, 2048, 11008]"
   t2 = prims.neg(t0)  # t2: "cuda:0 f32[2, 2048, 11008]"
   t3 = prims.exp(t2)  # t3: "cuda:0 f32[2, 2048, 11008]"
   t4 = prims.add(1.0, t3)  # t4: "cuda:0 f32[2, 2048, 11008]"
   t5 = prims.reciprocal(t4)  # t5: "cuda:0 f32[2, 2048, 11008]"
   t6 = prims.mul(t0, t5)  # t6: "cuda:0 f32[2, 2048, 11008]"
   t7 = prims.mul(t6, t1)  # t7: "cuda:0 f32[2, 2048, 11008]"
   t8 =